In [217]:

#Importing all the required libraries.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import folium 
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [219]:
df = pd.read_excel(r'C:\Users\kabilan.sundararaj\OneDrive - Accenture\Data Science\Final Assignment IBM Data Science prof\Neighborhood coordinates Bangalore.xlsx')
df.describe(include='all')

,Unnamed: 0,Unnamed: 1,Borough,Neighborhoods,Latitude,Longitude
count,0.0,64.000000,64,64,64.000000,64.000000
unique,NaN,NaN,8,64,NaN,NaN
top,NaN,NaN,Central,Ulsoor,NaN,NaN
freq,NaN,NaN,12,1,NaN,NaN
mean,NaN,31.500000,NaN,NaN,12.962340,77.601753
std,NaN,18.618987,NaN,NaN,0.059577,0.059817
min,NaN,0.000000,NaN,NaN,12.816800,77.482700
25%,NaN,15.750000,NaN,NaN,12.921100,77.555200
50%,NaN,31.500000,NaN,NaN,12.970845,77.593900
75%,NaN,47.250000,NaN,NaN,13.007200,77.639332


In [132]:
df.head(10)

,Unnamed: 0,Unnamed: 1,Borough,Neighborhoods,Latitude,Longitude
0,NaN,0,Central,Cantonment area,12.972442,77.580643
1,NaN,1,Central,Domlur,12.960992,77.638726
2,NaN,2,Central,Indiranagar,12.971891,77.641151
3,NaN,3,Central,Jeevanbheemanagar,12.962900,77.659500
4,NaN,4,Central,Malleswaram,13.003100,77.564300
5,NaN,5,Central,Pete area,12.962700,77.575800
6,NaN,6,Central,Rajajinagar,12.990100,77.552500
7,NaN,7,Central,Sadashivanagar,13.006800,77.581300
8,NaN,8,Central,Seshadripuram,12.993500,77.578700
9,NaN,9,Central,Shivajinagar,12.985700,77.605700


In [224]:
#df.drop('Unnamed: 0',axis=1,inplace=True)

In [225]:
CLIENT_ID = 'ADGTV1J0Z2MIRS5OP3MWGLRRXTVIGHBQXMPIVNWN00V1PZEC' 
CLIENT_SECRET = 'YGEHCLUWMO44NYD4LK1WNYXMIPZHSBO3BDCV4C3NSRCX5TCE' 
VERSION = '20180605'
LIMIT = 150

In [226]:
unique_boroughs_of_bangalore = df['Borough'].unique().tolist()

In [227]:

bangalore_latitude = df['Latitude'].mean()
bangalore_longitude = df['Longitude'].mean()
print("Latitude and Longitude of Bangalore are : ",bangalore_latitude,bangalore_longitude)

Latitude and Longitude of Bangalore are :  12.962339620312497 77.60175294687502


In [228]:
unique_boroughs_of_bangalore

['Central',
 'Eastern',
 'NorthEastern',
 'Northern',
 'SouthEastern',
 'Southern',
 'SouthernSuburbs',
 'Western']

In [229]:

borough_colors ={}
for i in unique_boroughs_of_bangalore:
    borough_colors[i] = '#%02X%02X%02X' % tuple(np.random.choice(range(256),size=3))

In [230]:
borough_colors

{'Central': '#C13DD2',
 'Eastern': '#462C45',
 'NorthEastern': '#57FC62',
 'Northern': '#F752EF',
 'SouthEastern': '#3645C7',
 'Southern': '#913A4D',
 'SouthernSuburbs': '#D1F7DB',
 'Western': '#DC95BB'}

In [231]:
bangalore_map = folium.Map(location=[bangalore_latitude,bangalore_longitude],zoom_start=12,control_scale=True)

In [232]:
for lat,lng,boro,nei in zip(df['Latitude'],
                           df['Longitude'],
                           df['Borough'],
                           df['Neighborhoods']):
    label_text = boro + ' - ' + nei
    label = folium.Popup(label_text,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    tooltip = label_text,
    radius = 4,
    popup = label,
    color=borough_colors[boro],
    fill=True,
    fill_color = borough_colors[boro],
    fill_opacity=0.7).add_to(bangalore_map)

In [87]:
bangalore_map

## Getting Venue data through Foursquare

In [233]:
def getNearbyVenues(names, boro, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, boro, lat, lng in zip(names, boro, latitudes, longitudes):
        print("Fetching venues for : ",name)            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            boro,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Borough',
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [234]:
bangalore_venues = getNearbyVenues(names=df['Neighborhoods'],latitudes=df['Latitude'],longitudes=df['Longitude'],boro=df['Borough'])

Fetching venues for :  Cantonment area
Fetching venues for :  Domlur
Fetching venues for :  Indiranagar
Fetching venues for :  Jeevanbheemanagar
Fetching venues for :  Malleswaram
Fetching venues for :  Pete area
Fetching venues for :  Rajajinagar
Fetching venues for :  Sadashivanagar
Fetching venues for :  Seshadripuram
Fetching venues for :  Shivajinagar
Fetching venues for :  Ulsoor
Fetching venues for :  Vasanth Nagar
Fetching venues for :  Bellandur
Fetching venues for :  CV Raman Nagar
Fetching venues for :  Hoodi
Fetching venues for :  Krishnarajapuram
Fetching venues for :  Mahadevapura
Fetching venues for :  Marathahalli
Fetching venues for :  Varthur
Fetching venues for :  Whitefield
Fetching venues for :  Banaswadi
Fetching venues for :  HBR Layout
Fetching venues for :  Horamavu
Fetching venues for :  Kammanahalli
Fetching venues for :  Lingarajapuram
Fetching venues for :  Ramamurthy Nagar
Fetching venues for :  Hebbal
Fetching venues for :  Jalahalli
Fetching venues for :

In [235]:
print("Total number of venues found in Bangalore are : ",bangalore_venues.shape[0])

Total number of venues found in Bangalore are :  590


In [236]:
bangalore_venues.head(5)

,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cantonment area,Central,12.972442,77.580643,Hotel Fishland,12.975569,77.578592,Seafood Restaurant
1,Cantonment area,Central,12.972442,77.580643,Adigas Hotel,12.973554,77.579161,Restaurant
2,Cantonment area,Central,12.972442,77.580643,Sapna Book House,12.976355,77.578461,Bookstore
3,Cantonment area,Central,12.972442,77.580643,Vasudev Adigas,12.973707,77.579257,Indian Restaurant
4,Cantonment area,Central,12.972442,77.580643,Sree Vishnu Bhavan,12.973950,77.579038,Indian Restaurant


# Neighborhoods of the two workspaces are Bellandur & Sadashivanagar

In [237]:
bangalore_venues.groupby('Neighborhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Anjanapura,1,1,1,1,1,1,1
Arekere,24,24,24,24,24,24,24
BTM Layout,36,36,36,36,36,36,36
Banashankari,11,11,11,11,11,11,11
Banaswadi,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...
Vidyaranyapura,1,1,1,1,1,1,1
Vijayanagar,4,4,4,4,4,4,4
Whitefield,9,9,9,9,9,9,9


## Analyze each neighborhood

In [238]:
# one hot encoding
Blr_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Blr_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Blr_onehot.columns[-1]] + list(Blr_onehot.columns[:-1])
Blr_onehot = Blr_onehot[fixed_columns]

Blr_onehot.head()

,Neighborhood,ATM,Andhra Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,...,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cantonment area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
Blr_onehot.shape

(590, 132)

In [240]:
Blr_grouped = Blr_onehot.groupby('Neighborhood').mean().reset_index()
Blr_grouped

,Neighborhood,ATM,Andhra Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,...,Tex-Mex Restaurant,Thai Restaurant,Toll Plaza,Toy / Game Store,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Anjanapura,1.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
1,Arekere,0.00,0.0,0.0,0.0,0.041667,0.0,0.0,0.041667,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
2,BTM Layout,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.027778,...,0.0,0.0,0.0,0.0,0.00,0.0,0.055556,0.0,0.0,0.0
3,Banashankari,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
4,Banaswadi,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.125000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Vidyaranyapura,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
59,Vijayanagar,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
60,Whitefield,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
61,Yelahanka,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0


In [241]:
Blr_grouped.shape

(63, 132)

## Finding top 30 common veneus for each neighborhood

In [242]:
# defining a function for creating a dataframe that lists the top venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [243]:
# creating a df that contains the top 30 venues for each neighborhood
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Blr_grouped['Neighborhood']

for ind in np.arange(Blr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Blr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(60)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Anjanapura,ATM,Department Store,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,...,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant
1,Arekere,Indian Restaurant,Sporting Goods Shop,Pizza Place,Gym / Fitness Center,Liquor Store,Fast Food Restaurant,Mughlai Restaurant,Department Store,Chinese Restaurant,...,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Art Museum,Bengali Restaurant,Gastropub
2,BTM Layout,Indian Restaurant,Ice Cream Shop,Chinese Restaurant,Snack Place,Bakery,Fast Food Restaurant,Pizza Place,Sandwich Place,Coffee Shop,...,Andhra Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Hookah Bar,Falafel Restaurant,Home Service,Fish & Chips Shop,Flea Market,Food & Drink Shop
3,Banashankari,Café,Indian Restaurant,Miscellaneous Shop,Pizza Place,North Indian Restaurant,Clothing Store,Fast Food Restaurant,Shoe Store,Men's Store,...,Historic Site,Halal Restaurant,Grocery Store,Gourmet Shop,Gym / Fitness Center,Gym,Flower Shop,Dessert Shop,Food & Drink Shop,Food Court
4,Banaswadi,Indian Restaurant,Bakery,Grocery Store,Market,Café,Pharmacy,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Donut Shop,...,Furniture / Home Store,Athletics & Sports,Gas Station,Gastropub,Gourmet Shop,Andhra Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
5,Basavanagudi,Indian Restaurant,Café,Hookah Bar,Restaurant,Fast Food Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hotel
6,Basaveshwaranagar,Fast Food Restaurant,Ice Cream Shop,Indian Restaurant,Pizza Place,Burger Joint,Snack Place,Sporting Goods Shop,Sandwich Place,Bakery,...,Dim Sum Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Food Court,Food Truck,Department Store,Furniture / Home Store,Gas Station,Gastropub
7,Begur,ATM,Bakery,Supermarket,Home Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Department Store,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hookah Bar,Hotel
8,Bellandur,Indian Restaurant,Fast Food Restaurant,Café,Kerala Restaurant,Bookstore,Multiplex,Coffee Shop,Pizza Place,Restaurant,...,Hookah Bar,Home Service,Falafel Restaurant,Historic Site,Fish & Chips Shop,Flea Market,Gym,Halal Restaurant,Grocery Store,Food & Drink Shop
9,Bommanahalli,Indian Restaurant,Athletics & Sports,Department Store,South Indian Restaurant,Hotel Bar,Hyderabadi Restaurant,Dim Sum Restaurant,Diner,Donut Shop,...,Furniture / Home Store,IT Services,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site


In [190]:
print(neighborhoods_venues_sorted.iloc[8],neighborhoods_venues_sorted.iloc[51])

Neighborhood                                Bellandur
1st Most Common Venue               Indian Restaurant
2nd Most Common Venue            Fast Food Restaurant
3rd Most Common Venue                            Café
4th Most Common Venue               Kerala Restaurant
5th Most Common Venue                       Bookstore
6th Most Common Venue                       Multiplex
7th Most Common Venue                     Coffee Shop
8th Most Common Venue                     Pizza Place
9th Most Common Venue                      Restaurant
10th Most Common Venue                  Shopping Mall
11th Most Common Venue                 Breakfast Spot
12th Most Common Venue                   Dessert Shop
13th Most Common Venue                      Juice Bar
14th Most Common Venue                         Bakery
15th Most Common Venue                       Tea Room
16th Most Common Venue                    Flower Shop
17th Most Common Venue                     Donut Shop
18th Most Common Venue    Ea

## Clustering neighborhoods

In [244]:
# set number of clusters
kclusters = 5

Blr_grouped_clustering = Blr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Blr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
np.unique(kmeans.labels_)


array([0, 1, 2, 3, 4])

In [245]:
df.drop('Unnamed: 1',axis=1,inplace=True)
df.set_index('Neighborhoods')

,Borough,Latitude,Longitude
Neighborhoods,,,
Cantonment area,Central,12.972442,77.580643
Domlur,Central,12.960992,77.638726
Indiranagar,Central,12.971891,77.641151
Jeevanbheemanagar,Central,12.962900,77.659500
Malleswaram,Central,13.003100,77.564300
...,...,...,...
Nagarbhavi,Western,12.959900,77.508300
Nandini Layout,Western,13.016000,77.533800
Nayandahalli,Western,12.941100,77.524800


In [246]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()
    

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,1,Anjanapura,ATM,Department Store,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,...,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant
1,3,Arekere,Indian Restaurant,Sporting Goods Shop,Pizza Place,Gym / Fitness Center,Liquor Store,Fast Food Restaurant,Mughlai Restaurant,Department Store,...,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Art Museum,Bengali Restaurant,Gastropub
2,3,BTM Layout,Indian Restaurant,Ice Cream Shop,Chinese Restaurant,Snack Place,Bakery,Fast Food Restaurant,Pizza Place,Sandwich Place,...,Andhra Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Hookah Bar,Falafel Restaurant,Home Service,Fish & Chips Shop,Flea Market,Food & Drink Shop
3,3,Banashankari,Café,Indian Restaurant,Miscellaneous Shop,Pizza Place,North Indian Restaurant,Clothing Store,Fast Food Restaurant,Shoe Store,...,Historic Site,Halal Restaurant,Grocery Store,Gourmet Shop,Gym / Fitness Center,Gym,Flower Shop,Dessert Shop,Food & Drink Shop,Food Court
4,0,Banaswadi,Indian Restaurant,Bakery,Grocery Store,Market,Café,Pharmacy,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,...,Furniture / Home Store,Athletics & Sports,Gas Station,Gastropub,Gourmet Shop,Andhra Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site


In [247]:
# add clustering labels

Blr_merged = df

# add latitude/longitude for each neighborhood
Blr_merged = Blr_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhoods')
Blr_merged['Cluster Labels'] = Blr_merged['Cluster Labels'].fillna(0).astype(int)
Blr_merged.head() # check the last columns!


,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Central,Cantonment area,12.972442,77.580643,0,Indian Restaurant,Restaurant,Bookstore,Seafood Restaurant,Food & Drink Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
1,Central,Domlur,12.960992,77.638726,0,Indian Restaurant,Café,BBQ Joint,Sandwich Place,Chinese Restaurant,...,Food Truck,Halal Restaurant,Hyderabadi Restaurant,Hotel Bar,Hotel,Hookah Bar,Home Service,Historic Site,Gym / Fitness Center,Dessert Shop
2,Central,Indiranagar,12.971891,77.641151,3,Pub,Lounge,Indian Restaurant,Café,Ice Cream Shop,...,Kids Store,Grocery Store,Hotel,Brewery,Chocolate Shop,Boutique,Bar,Vietnamese Restaurant,Udupi Restaurant,Toy / Game Store
3,Central,Jeevanbheemanagar,12.962900,77.659500,3,Fast Food Restaurant,Pizza Place,Gym,Food Court,Food & Drink Shop,...,Gastropub,Gourmet Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
4,Central,Malleswaram,13.003100,77.564300,3,Ice Cream Shop,Bakery,Breakfast Spot,Indian Restaurant,Asian Restaurant,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel


In [248]:
# create map
map_clusters = folium.Map(location=[bangalore_latitude,bangalore_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Blr_merged['Latitude'], Blr_merged['Longitude'], Blr_merged['Neighborhoods'], Blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

In [199]:
Blr_merged.loc[Blr_merged['Cluster Labels'] == 0, Blr_merged.columns[[1] + list(range(5, Blr_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
1,Domlur,Indian Restaurant,Café,BBQ Joint,Sandwich Place,Chinese Restaurant,Rajasthani Restaurant,Food & Drink Shop,Pizza Place,Asian Restaurant,...,Food Truck,Halal Restaurant,Hyderabadi Restaurant,Hotel Bar,Hotel,Hookah Bar,Home Service,Historic Site,Gym / Fitness Center,Dessert Shop
6,Rajajinagar,Indian Restaurant,Bakery,Paper / Office Supplies Store,Snack Place,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
7,Sadashivanagar,Coffee Shop,Indian Restaurant,Café,Department Store,Ice Cream Shop,Seafood Restaurant,Bookstore,Chinese Restaurant,Dessert Shop,...,Toll Plaza,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flower Shop
9,Shivajinagar,Indian Restaurant,Clothing Store,Men's Store,Tea Room,Donut Shop,Fast Food Restaurant,Market,Furniture / Home Store,South Indian Restaurant,...,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Home Service,French Restaurant,Department Store,Historic Site,Halal Restaurant
10,Ulsoor,Café,Bakery,Burger Joint,Yoga Studio,Flea Market,Food Truck,Food Court,Food & Drink Shop,Flower Shop,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel
12,Bellandur,Indian Restaurant,Fast Food Restaurant,Café,Kerala Restaurant,Bookstore,Multiplex,Coffee Shop,Pizza Place,Restaurant,...,Hookah Bar,Home Service,Falafel Restaurant,Historic Site,Fish & Chips Shop,Flea Market,Gym,Halal Restaurant,Grocery Store,Food & Drink Shop
13,CV Raman Nagar,Indian Restaurant,Pizza Place,Park,Department Store,Hyderabadi Restaurant,Food Court,Dessert Shop,Dim Sum Restaurant,Diner,...,Furniture / Home Store,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service
16,Mahadevapura,Indian Restaurant,Convenience Store,Shopping Mall,Bus Station,Fish & Chips Shop,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel
21,HBR Layout,Kitchen Supply Store,Coffee Shop,North Indian Restaurant,Café,Road,Fast Food Restaurant,Food & Drink Shop,Flower Shop,Flea Market,...,Furniture / Home Store,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service
26,Hebbal,Indian Restaurant,Market,Pizza Place,Coffee Shop,Park,Department Store,Home Service,Flower Shop,Dessert Shop,...,Food & Drink Shop,Historic Site,Food Court,Food Truck,Hotel,Furniture / Home Store,Hookah Bar,Gas Station,Gastropub,Gourmet Shop


In [200]:
Blr_merged.loc[Blr_merged['Cluster Labels'] == 1, Blr_merged.columns[[1] + list(range(5, Blr_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
3,Jeevanbheemanagar,Fast Food Restaurant,Pizza Place,Gym,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Falafel Restaurant,...,Gastropub,Gourmet Shop,Grocery Store,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
15,Krishnarajapuram,Market,Mobile Phone Shop,Motorcycle Shop,Bakery,Yoga Studio,French Restaurant,Food Court,Food & Drink Shop,Flower Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
17,Marathahalli,Clothing Store,Indian Restaurant,Movie Theater,Bakery,Fish & Chips Shop,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel
19,Whitefield,Café,Bakery,Gym / Fitness Center,Hotel Bar,Hotel,Kerala Restaurant,Eastern European Restaurant,Yoga Studio,Food & Drink Shop,...,French Restaurant,Furniture / Home Store,Dessert Shop,Gastropub,Gourmet Shop,Grocery Store,Gym,Halal Restaurant,Historic Site,Home Service
24,Lingarajapuram,Train Station,Yoga Studio,Fast Food Restaurant,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
30,R. T. Nagar,Café,Juice Bar,Grocery Store,Fast Food Restaurant,Bistro,Bus Station,Bakery,Resort,Yoga Studio,...,French Restaurant,Furniture / Home Store,Dim Sum Restaurant,Gastropub,Gourmet Shop,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service
37,Electronic City,Furniture / Home Store,Outlet Store,Bus Stop,Auto Garage,Toll Plaza,Fish & Chips Shop,Food Court,Food & Drink Shop,Flower Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
46,Kumaraswamy Layout,Burger Joint,Bakery,Breakfast Spot,Market,Sandwich Place,Café,Chinese Restaurant,Food Truck,Fast Food Restaurant,...,Furniture / Home Store,Dim Sum Restaurant,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
54,Kothnur,Women's Store,Athletics & Sports,Performing Arts Venue,Diner,Donut Shop,Dim Sum Restaurant,Eastern European Restaurant,Electronics Store,French Restaurant,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
61,Nayandahalli,Fast Food Restaurant,Road,Flea Market,Yoga Studio,French Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar


In [201]:
Blr_merged.loc[Blr_merged['Cluster Labels'] == 2, Blr_merged.columns[[1] + list(range(5, Blr_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Cantonment area,Indian Restaurant,Restaurant,Bookstore,Seafood Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
14,Hoodi,Indian Restaurant,Bus Station,French Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
18,Varthur,Supermarket,Yoga Studio,Fish & Chips Shop,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fast Food Restaurant,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar
27,Jalahalli,Indie Movie Theater,Playground,Convenience Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar
35,Bommasandra,Indian Restaurant,Department Store,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,...,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant
42,Basavanagudi,Indian Restaurant,Café,Hookah Bar,Restaurant,Fast Food Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hotel
52,Gottigere,Indian Restaurant,Department Store,Pharmacy,Art Museum,Furniture / Home Store,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel
60,Nandini Layout,Vegetarian / Vegan Restaurant,Yoga Studio,Fast Food Restaurant,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,...,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar


In [202]:
Blr_merged.loc[Blr_merged['Cluster Labels'] == 3, Blr_merged.columns[[1] + list(range(5, Blr_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
2,Indiranagar,Pub,Lounge,Indian Restaurant,Café,Ice Cream Shop,Cocktail Bar,Restaurant,Gym,Italian Restaurant,...,Kids Store,Grocery Store,Hotel,Brewery,Chocolate Shop,Boutique,Bar,Vietnamese Restaurant,Udupi Restaurant,Toy / Game Store
5,Pete area,Indian Restaurant,Flower Shop,Market,Historic Site,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,French Restaurant,...,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Home Service,Hookah Bar,Hotel,Hotel Bar
8,Seshadripuram,Indian Restaurant,Sushi Restaurant,Food & Drink Shop,Asian Restaurant,Hotel,Lounge,Chaat Place,Chinese Restaurant,Flower Shop,...,French Restaurant,Furniture / Home Store,Dessert Shop,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
11,Vasanth Nagar,Indian Restaurant,Coffee Shop,Chinese Restaurant,Italian Restaurant,Hotel,Pizza Place,Steakhouse,Mediterranean Restaurant,French Restaurant,...,Flower Shop,Eastern European Restaurant,Donut Shop,Asian Restaurant,Food Truck,Food Court,Dim Sum Restaurant,Furniture / Home Store,Gas Station,Gastropub
20,Banaswadi,Indian Restaurant,Bakery,Grocery Store,Market,Café,Pharmacy,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Donut Shop,...,Furniture / Home Store,Athletics & Sports,Gas Station,Gastropub,Gourmet Shop,Andhra Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
23,Kammanahalli,Indian Restaurant,Fast Food Restaurant,Snack Place,Market,Chinese Restaurant,Korean Restaurant,Kerala Restaurant,Burger Joint,Falafel Restaurant,...,Gas Station,Furniture / Home Store,Dim Sum Restaurant,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
33,Yeshwanthpur,Indian Restaurant,Bar,Restaurant,Seafood Restaurant,Light Rail Station,Mediterranean Restaurant,Hotel,Historic Site,Food & Drink Shop,...,Food Court,Halal Restaurant,Food Truck,French Restaurant,Department Store,Furniture / Home Store,Gas Station,Gastropub,Hookah Bar,Gourmet Shop
34,Bommanahalli,Indian Restaurant,Athletics & Sports,Department Store,South Indian Restaurant,Hotel Bar,Hyderabadi Restaurant,Dim Sum Restaurant,Diner,Donut Shop,...,Furniture / Home Store,IT Services,Gas Station,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site
36,BTM Layout,Indian Restaurant,Ice Cream Shop,Chinese Restaurant,Snack Place,Bakery,Fast Food Restaurant,Pizza Place,Sandwich Place,Coffee Shop,...,Andhra Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Hookah Bar,Falafel Restaurant,Home Service,Fish & Chips Shop,Flea Market,Food & Drink Shop
38,HSR Layout,Department Store,Bakery,Park,Pizza Place,Punjabi Restaurant,Fish & Chips Shop,Chinese Restaurant,Café,Gastropub,...,Hotel,Fast Food Restaurant,Hookah Bar,Home Service,Historic Site,Gym / Fitness Center,Flower Shop,Food & Drink Shop,Food Court,Food Truck


In [187]:
Blr_merged.loc[Blr_merged['Cluster Labels'] == 5, Blr_merged.columns[[1] + list(range(5, Blr_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Finding the most similar neighborhoods in cluster 0 

## the neighborhoods of existing 2 workspaces fall under this category

In [208]:
Selected_neighborhoods = Blr_merged.loc[Blr_merged['Cluster Labels'] == 0,Blr_merged.columns[1]]
Selected_neighborhoods.head()

1             Domlur
6        Rajajinagar
7     Sadashivanagar
9       Shivajinagar
10            Ulsoor
Name: Neighborhoods, dtype: object

,Neighborhood,ATM,Andhra Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,...,Thai Restaurant,Toll Plaza,Toy / Game Store,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster
0,Anjanapura,1.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,5
1,Arekere,0.00,0.0,0.0,0.0,0.041667,0.0,0.0,0.041667,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,4
2,BTM Layout,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.027778,...,0.0,0.0,0.0,0.00,0.0,0.055556,0.0,0.0,0.0,4
3,Banashankari,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,2
4,Banaswadi,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.125000,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Vidyaranyapura,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,13
59,Vijayanagar,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,4
60,Whitefield,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,17
61,Yelahanka,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0


In [249]:
#Exporting top venues df 
writer = pd.ExcelWriter(r'C:\Users\kabilan.sundararaj\OneDrive - Accenture\Data Science\Final Assignment IBM Data Science prof\Final_Options.xlsx', engine='xlsxwriter')
Blr_grouped.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [214]:
#Exporting selected neighborhoods
writer = pd.ExcelWriter(r'C:\Users\kabilan.sundararaj\OneDrive - Accenture\Data Science\Final Assignment IBM Data Science prof\Final_Options.xlsx', engine='xlsxwriter')
Selected_neighborhoods.to_excel(writer, sheet_name='Selected_neighborhoods')
writer.save()

In [216]:
# Similarity scores for neighborhoods in selected cluster is calculated  & sorted in Top_Option xlsx